In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from datetime import datetime

from __future__ import print_function
import pickle

In [2]:
url = 'https://www.the-numbers.com/movie/budgets/all/901'

user_agent = {'User-agent': 'Mozilla/5.0'}

response  = requests.get(url, headers = user_agent)

page = response.text

soup = BeautifulSoup(page,"lxml")

In [3]:
table = soup.find('table')
table

<table>
<tr><th> </th><th>Release<br/>Date</th><th>Movie</th><th>Production<br/>Budget</th><th>Domestic<br/>Gross</th><th>Worldwide<br/>Gross</th></tr>
<tr><td class="data">901</td>
<td><a href="/box-office-chart/daily/2002/04/18">Apr 18, 2002</a></td>
<td><b><a href="/movie/Scorpion-King-The#tab=summary">The Scorpion King</a></b></td>
<td class="data"> $60,000,000</td>
<td class="data"> $90,580,000</td>
<td class="data"> $165,890,634</td>
</tr>
<tr><td class="data">902</td>
<td><a href="/box-office-chart/daily/2010/10/15">Oct 15, 2010</a></td>
<td><b><a href="/movie/Red-(2010)#tab=summary">Red</a></b></td>
<td class="data"> $60,000,000</td>
<td class="data"> $90,380,162</td>
<td class="data"> $196,439,693</td>
</tr>
<tr><td class="data">903</td>
<td><a href="/box-office-chart/daily/2004/03/05">Mar 5, 2004</a></td>
<td><b><a href="/movie/Starsky-and-Hutch#tab=summary">Starsky &amp; Hutch</a></b></td>
<td class="data"> $60,000,000</td>
<td class="data"> $88,200,225</td>
<td class="data"

In [4]:
rows = [row for row in table.find_all('tr')]

In [5]:
movies = {}

for row in rows[1:]:
    items = row.find_all('td')
    link = row.find('a')
    title, url = link.text, link['href']
    movies[title] = [url] + [i.text for i in items]
    
movies

{'Apr 18, 2002': ['/box-office-chart/daily/2002/04/18',
  '901',
  'Apr 18, 2002',
  'The Scorpion King',
  '\xa0$60,000,000',
  '\xa0$90,580,000',
  '\xa0$165,890,634'],
 'Oct 15, 2010': ['/box-office-chart/daily/2010/10/15',
  '902',
  'Oct 15, 2010',
  'Red',
  '\xa0$60,000,000',
  '\xa0$90,380,162',
  '\xa0$196,439,693'],
 'Mar 5, 2004': ['/box-office-chart/daily/2004/03/05',
  '903',
  'Mar 5, 2004',
  'Starsky & Hutch',
  '\xa0$60,000,000',
  '\xa0$88,200,225',
  '\xa0$170,200,225'],
 'Jun 27, 1990': ['/box-office-chart/daily/1990/06/27',
  '904',
  'Jun 27, 1990',
  'Days of Thunder',
  '\xa0$60,000,000',
  '\xa0$82,670,733',
  '\xa0$157,670,733'],
 'Dec 21, 2005': ['/box-office-chart/daily/2005/12/21',
  '905',
  'Dec 21, 2005',
  'Cheaper by the Dozen 2',
  '\xa0$60,000,000',
  '\xa0$82,571,173',
  '\xa0$135,015,330'],
 'Aug 13, 2010': ['/box-office-chart/daily/2010/08/13',
  '906',
  'Aug 13, 2010',
  'Eat Pray Love',
  '\xa0$60,000,000',
  '\xa0$80,574,010',
  '\xa0$206,598,

In [6]:
budget = pd.DataFrame(movies).T  #transpose
budget.columns = ['link_stub','rank_all_movies','release','title', 
                    'budget', 'domestic_gross','worldwide_gross']

budget.shape

(100, 7)

In [7]:
budget['budget'] = budget['budget'].str.replace('$', '')
budget['budget'] = budget['budget'].str.replace(',', '')
budget['budget'] = budget['budget'].astype(int)

In [8]:
budget.head()

,link_stub,rank_all_movies,release,title,budget,domestic_gross,worldwide_gross
"Apr 18, 2002",/box-office-chart/daily/2002/04/18,901,"Apr 18, 2002",The Scorpion King,60000000,"$90,580,000","$165,890,634"
"Oct 15, 2010",/box-office-chart/daily/2010/10/15,902,"Oct 15, 2010",Red,60000000,"$90,380,162","$196,439,693"
"Mar 5, 2004",/box-office-chart/daily/2004/03/05,903,"Mar 5, 2004",Starsky & Hutch,60000000,"$88,200,225","$170,200,225"
"Jun 27, 1990",/box-office-chart/daily/1990/06/27,904,"Jun 27, 1990",Days of Thunder,60000000,"$82,670,733","$157,670,733"
"Dec 21, 2005",/box-office-chart/daily/2005/12/21,905,"Dec 21, 2005",Cheaper by the Dozen 2,60000000,"$82,571,173","$135,015,330"


In [9]:
budget.to_csv('budget10.csv')